In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
from PIL import Image
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import hog
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the path to your image folder in Google Drive
image_folder_path = '/content/drive/MyDrive/Machine learning lab/Machine learning project/Dataset'

In [ ]:
# Image dimensions
img_width, img_height = 128, 128

In [ ]:
# Function to load images and labels
def load_images_and_labels(data_dir):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    file_path = os.path.join(label_dir, filename)
                    try:
                        img = Image.open(file_path)
                        if img.mode != 'RGB':
                            img = img.convert('RGB')
                        img = img.resize((img_width, img_height))
                        img_array = np.array(img) / 255.0 # Normalize pixel values
                        images.append(img_array)
                        labels.append(label)
                    except Exception as e:
                        print(f"Error processing {filename}: {e}")
    return np.array(images), np.array(labels)

In [ ]:
# Load data
images, labels = load_images_and_labels(image_folder_path)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)
one_hot_labels = to_categorical(encoded_labels, num_classes=num_classes)

In [ ]:
# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(images, one_hot_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

# --- Conventional Machine Learning (HOG + SVM)

In [ ]:
# HOG Feature Extraction
def extract_hog_features(images):
    features = []
    for img_array in images:
        fd = hog(img_array, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), multichannel=True)
        features.append(fd)
    return np.array(features)

In [ ]:
!pip install --upgrade scikit-image>=0.19

In [ ]:
from skimage.feature import hog
from skimage.color import rgb2gray
import numpy as np

def extract_hog_features(images, pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=9):
    """
    Extract HOG features for a set of images.

    Parameters:
    - images: List or array of images (each image should be a 2D or 3D array).
    - pixels_per_cell: Size of a cell in pixels (tuple of two integers).
    - cells_per_block: Number of cells in each block (tuple of two integers).
    - orientations: Number of orientation bins.

    Returns:
    - features: Numpy array of HOG feature vectors for all images.
    """
    features = []
    for img in images:
        # Convert to grayscale if the image is RGB
        if img.ndim == 3:
            img = rgb2gray(img)
        # Extract HOG features
        hog_features = hog(
            img,
            orientations=orientations,
            pixels_per_cell=pixels_per_cell,
            cells_per_block=cells_per_block,
            block_norm='L2-Hys',
            transform_sqrt=True,
            feature_vector=True
        )
        features.append(hog_features)
    return np.array(features)

# Example usage:
# Assuming X_train, X_val, and X_test are arrays of images (e.g., numpy arrays)
X_train_hog = extract_hog_features(X_train)
X_val_hog = extract_hog_features(X_val)
X_test_hog = extract_hog_features(X_test)


In [ ]:
# Convert y_train, y_val, y_test to numerical labels
y_train_numerical = np.argmax(y_train, axis=1)
y_val_numerical = np.argmax(y_val, axis=1)
y_test_numerical = np.argmax(y_test, axis=1)

In [ ]:
# Train an SVM classifier
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(X_train_hog, y_train_numerical)

SVC(kernel='linear', probability=True)

In [ ]:
# Predict and evaluate on the validation set
y_val_pred = svm_classifier.predict(X_val_hog)
print("Validation Accuracy (SVM):", accuracy_score(y_val_numerical, y_val_pred))
print(classification_report(y_val_numerical, y_val_pred, target_names=label_encoder.classes_))

Validation Accuracy (SVM): 0.7631578947368421
              precision    recall  f1-score   support

         Bus       0.71      0.75      0.73        20
         Car       0.85      0.81      0.83        21
       Truck       0.62      0.53      0.57        19
  motorcycle       0.84      1.00      0.91        16

    accuracy                           0.76        76
   macro avg       0.76      0.77      0.76        76
weighted avg       0.76      0.76      0.76        76



In [ ]:
# Predict and evaluate on the test set
y_test_pred = svm_classifier.predict(X_test_hog)
print("Test Accuracy (SVM):", accuracy_score(y_test_numerical, y_test_pred))
print(classification_report(y_test_numerical, y_test_pred, target_names=label_encoder.classes_))

Test Accuracy (SVM): 0.6447368421052632
              precision    recall  f1-score   support

         Bus       0.65      0.65      0.65        20
         Car       0.70      0.58      0.64        12
       Truck       0.56      0.56      0.56        27
  motorcycle       0.74      0.82      0.78        17

    accuracy                           0.64        76
   macro avg       0.66      0.65      0.65        76
weighted avg       0.64      0.64      0.64        76



# Convolutional Neural Network (CNN)

In [ ]:
# CNN Model Definition
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.3232 - loss: 2.0109 - val_accuracy: 0.4079 - val_loss: 1.3692
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.4208 - loss: 1.3210 - val_accuracy: 0.3947 - val_loss: 1.3310
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.4155 - loss: 1.2296 - val_accuracy: 0.4342 - val_loss: 1.3841
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.4820 - loss: 1.1044 - val_accuracy: 0.4342 - val_loss: 1.5009
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6371 - loss: 0.8879 - val_accuracy: 0.5000 - val_loss: 1.0527
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.7288 - loss: 0.6351 - val_accuracy: 0.5526 - val_loss: 1.1055
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.7966 - loss: 0.5757 - val_accuracy: 0.5658 - val_loss: 1.1967
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.8905 - loss: 0.2998 - val_accuracy: 0.6184 - val_loss: 1.2881
Epoch 9/

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy (CNN): {accuracy}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 358ms/step - accuracy: 0.5144 - loss: 2.8832
Test Accuracy (CNN): 0.5131579041481018


In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 398ms/step


In [ ]:
# Generate and print classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

         Bus       0.42      0.75      0.54        20
         Car       0.44      0.33      0.38        12
       Truck       0.62      0.37      0.47        27
  motorcycle       0.67      0.59      0.62        17

    accuracy                           0.51        76
   macro avg       0.54      0.51      0.50        76
weighted avg       0.55      0.51      0.51        76

